In [ ]:
한글은 똑같은데
Okt(한글형태소분석기) 로 토큰화하기만 하면 나머지는 똑같음

OKt => Vectorizer => Model


'교도소 이야기구먼 솔직히 재미는 없다' 를 okt에 넣으면
['교도소','이야기','구먼','솔직하다']... 이런식으로 나옴
이걸 Vectorizer에 집어넣어야 하는데 바로 집어넣을 수가 없음
스트링의 리스트로 만들어진 걸
['스트링'] 으로 만든 다음에
['교도소 이야기 구먼 솔직하다 재미 는 없다 평점'] <== 이런 식으로
벡터라이저로 만들어야 함

모델을 이제 해야하는데
이떄까지 우린 주로 logistic regression, svm, randome forest 정도를 했음
근데 이제 문자를 할 때는 Naive Bayes로 해야함

그래서 오늘은 count + lr, naive, tfidf + lr, naive 총 4개를 해야 함

요게 옜날에 했던 것들이 있는 거임

이거 할 때는 모델 로딩하는거 생략하세요..

In [2]:
import joblib
import re
import pandas as pd 

모델 가져오기

In [3]:
model_cl = joblib.load('../static/model/hangul_count_lr.pkl')
model_cn = joblib.load('../static/model/hangul_count_nb.pkl')
model_tl = joblib.load('../static/model/hangul_tfidf_lr.pkl')
model_tn = joblib.load('../static/model/hangul_tfidf_nb.pkl')

movie_test, movie_train 가져와야 인덱스로 받을 때 label을 얻을 수 있음

In [4]:
train_df = pd.read_table('../static/data/advanced/movie_train.tsv')
del train_df['Unnamed: 0']
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [5]:
test_df = pd.read_table('../static/data/advanced/movie_test.tsv')
del test_df['Unnamed: 0']
test_df.head(3)

,id,document,label
0,6270596,굳 ㅋ,1
1,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
2,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0


In [6]:
test_df[test_df.label == 1].head(25)

,id,document,label
0,6270596,굳 ㅋ,1
4,7898805,음악이 주가 된 최고의 음악영화,1
8,6242223,이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 모든 사람이 그렇지는 않네,1
9,7462111,괜찮네요오랜만포켓몬스터잼밌어요,1
11,6900881,청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다 찰나의 아름다움을 잘 포착한 섬...,1
12,9629375,눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다,1
14,10268521,소위 ㅈ문가라는 평점은 뭐냐,1
15,2968565,최고,1
17,6406912,나이스,1
19,9305768,일의 금요일 나이트메어시리즈와 함께 가장 많은 시리즈를 양산해냈던 헬레이저 시리즈의...,1


테스트 셋 데이터 인덱스로 가져올 때

In [7]:
index = 47

In [8]:
review = test_df.iloc[index,1]
print(review)
label = test_df.label[index]

이런스타일 액션영화 굿굿


클라이언트에서 직접 받을 때

In [9]:
""" review = "진짜 개노잼이다... 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ"  """

' review = "진짜 개노잼이다... 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ"  '

In [10]:
""" review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review)  """

" review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', review)  "

In [11]:
from konlpy.tag import Okt
okt = Okt()

In [12]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
morph = okt.morphs(review, stem = True) # 이걸 넣어서 안 됐네...
# 트레인셋은 전처리만 된 거지 토큰화는 되지 않았다.
# X_test는 토큰화가 되긴 했지만 그걸 저장하지 않았고
# 내가 위에서 불러운 test_df는 토큰화가 되지 않은 생 df였다.
temp_X = ' '.join([word for word in morph if not word in stopwords])
temp_X = [temp_X]
temp_X

['이런 스타일 액션영화 굿굿']

In [13]:
pred_cl = model_cl.predict(temp_X)
pred_cn = model_cn.predict(temp_X)
pred_tl = model_tl.predict(temp_X)
pred_tn = model_tn.predict(temp_X)

In [14]:
print(label,pred_cl[0], pred_cn[0], pred_tl[0], pred_tn[0])

1 1 1 1 1


In [15]:
""" from tqdm import tqdm
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
for i in tqdm(range(3000,6000)):
    review = test_df.iloc[index,1]
    label = test_df.label[index]
    temp_X = ' '.join([word for word in review if not word in stopwords])
    temp_X = [temp_X]
    pred_cl = model_cl.predict(temp_X)
    pred_cn = model_cn.predict(temp_X)
    pred_tl = model_tl.predict(temp_X)
    pred_tn = model_tn.predict(temp_X)
    if pred_cl[0] == 0:
        continue
    print(label,pred_cl[0], pred_cn[0], pred_tl[0], pred_tn[0]) """

" from tqdm import tqdm\nstopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']\nfor i in tqdm(range(3000,6000)):\n    review = test_df.iloc[index,1]\n    label = test_df.label[index]\n    temp_X = ' '.join([word for word in review if not word in stopwords])\n    temp_X = [temp_X]\n    pred_cl = model_cl.predict(temp_X)\n    pred_cn = model_cn.predict(temp_X)\n    pred_tl = model_tl.predict(temp_X)\n    pred_tn = model_tn.predict(temp_X)\n    if pred_cl[0] == 0:\n        continue\n    print(label,pred_cl[0], pred_cn[0], pred_tl[0], pred_tn[0]) "